In [1]:
print("hi")

hi


In [3]:
secret_message = "skrrt"

In [ ]:
https://jupyter.icos-cp.eu/user/elias-ervela/?token=fe7f822e64a64521a2573e4ebb71bafd

In [616]:
#v2

import cluster as clt
from tqdm.notebook import tqdm # progress bar


# Reset the dataframe
TNO_MACC_III_point_sources_NOX.loc[:, "Times found"] = 0
TNO_MACC_III_point_sources_NOX.loc[:, "Times missed"] = 0
TNO_MACC_III_point_sources_NOX.loc[:, "Found in files"] = ""
TNO_MACC_III_point_sources_NOX.loc[:, "Missed in files"] = ""

my_clusters_per_file_v2 = []
number_of_facilities_in_plumes_per_file = []

## Get facility points
facility_points = TNO_MACC_III_point_sources_NOX[["Lon", "Lat"]]

# Go through every file
for i in tqdm(range(len(SMARTCARB_filenames_all))):
    # Get the SMARTCARB data from files and save the data to lon, lat and no2
    filename = os.path.join(SMARTCARB_DATA_PATH, SMARTCARB_filenames_all[i])
    data = ddeq.smartcarb.read_level2(
        filename,
        co2_noise_scenario=None,
        no2_noise_scenario='low',
        co_noise_scenario=None,
        only_observations=True,
    )
    lon_m = np.array(data["lon"])
    lat_m = np.array(data["lat"])
    no2_m = np.array(data["NO2"])
    
    #no2_m_padded = np.pad(no2_m, pad_width=1, mode='constant', constant_values=np.nan)
    
    # Remove points outside domain
    domain_extent=[8.30719191586997, 18.170863673993093, 49.978223611443795, 54.694193446535714]
    area_boolean = np.logical_and(np.logical_and(lon_m>=domain_extent[0], lon_m<=domain_extent[1]), np.logical_and(lat_m>=domain_extent[2], lat_m<=domain_extent[3]))
    no2_m[~area_boolean] = np.nan
    
    lon = lon_m.flatten()
    lat = lat_m.flatten()
    no2 = no2_m.flatten()
    
    #nan_indices = np.where(np.isnan(no2))
    not_nan = np.logical_not(np.isnan(no2))
    
    lon_notNaN = lon[not_nan]
    lat_notNaN = lat[not_nan]
    no2_notNaN = no2[not_nan]
    lon_NaN = lon[~not_nan]
    lat_NaN = lat[~not_nan]
    
    
    # Continue if there are no points in the file
    if len(no2_notNaN) == 0: 
        number_of_facilities_in_plumes_per_file.append([])
        continue    

    # Plume detection
    clusters = clt.density_based_clustering_with_varying_radius(
        np.transpose([lon_notNaN, lat_notNaN]),
        no2_notNaN,
        radius_func="default",
        n_clusters="auto",
        point_value_threshold="stds_from_median",
        stds=3.3,
        distance_matrix="euclidean",
        radius_func_sigmas_threshold=1.3,
        max_points_in_start_radius=5,
        local_box_size=3,
        verbose=True,
    )

    # Check if there are facilities in the plumes

    ## Take only facilities that are near any points in the dataset
    is_near_any_points = np.any(
        haversine_distances(
            np.radians(facility_points),
            np.radians(np.transpose([lon_notNaN, lat_notNaN])),
        )
        * 6371
        <= 5,
        axis=1,
    )
    facilities_in_file = facility_points[is_near_any_points]

    ## Initialise
    is_facility_found = np.zeros(len(facilities_in_file), dtype=bool)
    number_of_facilities_in_plumes = np.zeros(len(np.unique(clusters))-1)
    
    ## Go through every cluster
    for j in np.unique(clusters):
        j = int(j)
        if j == 0: continue
        
        lon_notNaN_cluster = lon_notNaN[clusters == j]
        lat_notNaN_cluster = lat_notNaN[clusters == j]

        facility_points_in_dataset = facilities_in_file[["Lon", "Lat"]].values

        ### Is a facility near a plume?
        is_facility_near_any_plume_points = np.any(
            haversine_distances(
                np.radians(facility_points_in_dataset),
                np.radians(np.transpose([lon_notNaN_cluster, lat_notNaN_cluster])),
            )
            * 6371
            <= 5,
            axis=1,
        )
        
        ### If yes, update the dataframe
        is_facility_found[is_facility_near_any_plume_points] = True
        
        ### Update the number of facilities in the plume
        number_of_facilities_in_plumes[j-1] = is_facility_near_any_plume_points.sum()
        
    ## Update the dataframe for facilities, if they were found or missed
    index_of_found_facilities = facilities_in_file[is_facility_found].index
    TNO_MACC_III_point_sources_NOX.loc[index_of_found_facilities, "Times found"] += 1
    TNO_MACC_III_point_sources_NOX.loc[index_of_found_facilities, "Found in files"] += f"{i},"
    index_of_missed_facilities = facilities_in_file[~is_facility_found].index
    TNO_MACC_III_point_sources_NOX.loc[index_of_missed_facilities, "Times missed"] += 1
    TNO_MACC_III_point_sources_NOX.loc[index_of_missed_facilities, "Missed in files"] += f"{i},"
      
        
    ## Turn the plume clusters array in to matrix shape to make the next operation faster
    clusters_with_nans = np.empty_like(no2)
    clusters_with_nans[~not_nan] = np.nan
    clusters_with_nans[not_nan] = clusters
    clusters_m = clusters_with_nans.reshape(lon_m.shape)
    clusters_m_padded = np.pad(clusters_m, pad_width=1, mode='constant', constant_values=np.nan)
        
    ## If there were no facilities in the plume, check if there were nan values next to the plume
    for j in np.unique(clusters):
        j = int(j)
        if j == 0: continue
        if number_of_facilities_in_plumes[j-1] == 0:
            
            is_adjacent = adjacent_cells(matrix=clusters_m_padded, number=j)
            
            ### Is a nan value near a plume?
            if np.isnan(clusters_m_padded[is_adjacent]).sum() > 0:
                ### If yes, marked the plume as dismissed, with "-1"
                number_of_facilities_in_plumes[j-1] = -1

            
            
    number_of_facilities_in_plumes_per_file.append(number_of_facilities_in_plumes)
   
   
 
def adjacent_cells(matrix, number):
    rows, cols = len(matrix), len(matrix[0])
    result = [[False] * cols for _ in range(rows)]
    for i in range(rows):
        for j in range(cols):
            if matrix[i][j] == number:
                result[i][j] = True
                if i > 0:
                    result[i - 1][j] = True
                    if j > 0:
                        result[i - 1][j - 1] = True
                    if j < cols - 1:
                        result[i - 1][j + 1] = True
                if i < rows - 1:
                    result[i + 1][j] = True
                    if j > 0:
                        result[i + 1][j - 1] = True
                    if j < cols - 1:
                        result[i + 1][j + 1] = True
                if j > 0:
                    result[i][j - 1] = True
                if j < cols - 1:
                    result[i][j + 1] = True
    return result

  0%|          | 0/6 [00:00<?, ?it/s]

Found  13  clusters
Found  17  clusters
Found  10  clusters
No clusters found. All points are too close to the mean. Consider lowering stds. Currently stds=3.300000.
